<a href="https://colab.research.google.com/github/laNaranjaja/GenAI/blob/main/Clase_Streamlit_Chat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install streamlit --quiet
!pip install langchain --quiet
!pip install google-search-results --quiet
!pip install langchain_experimental --quiet
!pip install arxiv --quiet
!pip install streamlit_chat --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.9/815.9 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.0/167.0 kB 1.7 MB/s eta 

In [ ]:
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
added 22 packages from 22 contributors and audited 22 packages in 2.385s

3 packages are looking for funding
  run `npm fund` for details

found 1 moderate severity vulnerability
  run `npm audit fix` to fix them, or `npm audit` for details


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%%writefile app.py

import os
import json
import streamlit as st
from langchain.chat_models import ChatVertexAI
from langchain.utilities import SerpAPIWrapper
from langchain.agents import initialize_agent
from langchain.agents import Tool
from langchain.agents import AgentType
from langchain.memory import ConversationBufferMemory
from streamlit_chat import message

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'PATH A TU JSON DE GCP'
os.environ['SERPAPI_API_KEY'] = 'TU API TOKEN DE SERPAPI'

chat = ChatVertexAI(max_output_tokens=2048, temperature=0)
search = SerpAPIWrapper()

tools = [
    Tool(
        name="Current Search",
        func=search.run,
        description="Useful for when you need to answer questions about current events or the current state of the world"
    )
]

memory = ConversationBufferMemory(memory_key="chat_history")

agent_chain = initialize_agent(
    tools,
    chat,
    handle_parsing_errors=True,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    verbose=True,
    memory=memory
)

st.title('My PaLM Chat :sunglasses:')

if 'history' not in st.session_state:
  st.session_state['history'] = []

if 'generated' not in st.session_state:
  st.session_state['generated'] = ["Hello! Ask me anything!🤖"]

if 'past' not in st.session_state:
  st.session_state['past'] = ["Hey! 👋"]

response_container = st.container()
container = st.container()

with container:
  with st.form(key='my_form', clear_on_submit=True):
    user_input = st.text_input("Query:", placeholder="Put your question here", key='input')
    submit_button = st.form_submit_button(label='Preguntar')

  if submit_button:
    output = agent_chain.run(input=user_input)
    st.session_state['past'].append(user_input)
    st.session_state['generated'].append(output)
    st.session_state['history'].append((user_input,output))

if st.session_state['generated']:
  with response_container:
    for i in range(len(st.session_state['generated'])):
      message(st.session_state["past"][i], is_user=True, key=str(i) + '_user', avatar_style="big-smile")
      message(st.session_state["generated"][i], key=str(i), avatar_style="thumbs")

Overwriting app.py


In [ ]:
!streamlit run app.py &>/content/logs.txt &

In [ ]:
import urllib
print("Password/Enpoint IP for localtunnel is:",urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))

Password/Enpoint IP for localtunnel is: 104.198.68.3


In [ ]:
!npx localtunnel --port 8501

npx: installed 22 in 2.45s
your url is: https://mean-nails-swim.loca.lt
^C
